<a href="https://colab.research.google.com/github/jessica8403/TIL/blob/master/Resnet50V2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model,Sequential
import zipfile

import gdown

In [ ]:
train_path = '/content/drive/MyDrive/멀티캠퍼스/data/train/'

In [ ]:
# data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator



Hyper parameter 수정

In [ ]:
# 예측할 클래스 수
classes = 10

# Input으로 사용될 크기와 채널수
height = 256
width = 256
channels = 3

이미지 처리 진행

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255, validation_split=0.2)
image_data_train = image_generator.flow_from_directory(train_path,subset='training')
image_data_test = image_generator.flow_from_directory(train_path,subset='validation')

Found 40000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [ ]:
image_generator = ImageDataGenerator(
    rotation_range=30,
    brightness_range = [0.8, 1.0],
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

모델로드

In [ ]:
resnetv2 = tf.keras.applications.ResNet50V2(include_top=False,input_shape=(height,width,channels))

94683136/94668760 [==============================] - 1s 0us/step


In [ ]:
resnetv2.summary()

In [ ]:
resnetv2.trainable=False

기존 레이어 학습 동결 

In [ ]:
model = Sequential([
                 resnetv2,
                 Dense(512,activation='relu'),
                 BatchNormalization(),
                 GlobalAveragePooling2D(),
                 Dense(classes,activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 dense (Dense)               (None, 8, 8, 512)         1049088   
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 512)        2048      
 ormalization)                                                   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,621,066
Trainable params: 1,055,242
Non

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [ ]:
def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
lrs = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
hist=model.fit(image_data_train,batch_size=32,epochs=10,callbacks=[lrs],validation_data=(image_data_test),
          validation_steps =image_data_test.samples/image_data_test.batch_size)

Epoch 1/10
  78/1250 [>.............................] - ETA: 3:41:50 - loss: 0.9707 - accuracy: 0.6599

KeyboardInterrupt: ignored

In [ ]:
plt.plot( hist.history['loss'], label='train loss')
plt.plot( hist.history['val_loss'], label='validation loss')
plt.legend()



In [ ]:
plt.plot( hist.history['acc'], label='train accuracy')
plt.plot( hist.history['val_acc'], label='validation accuracy')
plt.legend()